# Tutorial: Linear Programming, (CPLEX Part 1)
This notebook is part of **[Prescriptive Analytics for Python](http://ibmdecisionoptimization.github.io/docplex-doc/)**

## What is a Linear expression?

**核心要点：**

*   **线性表达式的要求：** 只能包含变量的常数倍数（例如：2x, -5y, 0.3z）和常数项（例如：5, -10, 3.14）。
*   **不允许出现的项：**
    *   **变量相乘：** 两个或多个变量相乘的项（如：x\*y, a\*b\*c, xz）
    *   **高阶项：** 变量的平方、立方或更高次方的项（如：x², y³, z⁴）
    *   **指数：** 变量作为指数的项（如：2ˣ, eʸ）
    *   **对数：** 变量在对数函数中的项（如：log(x), ln(y)）
    *   **绝对值：** 变量在绝对值符号内的项（如：|x|, |y-2|）


**什么是线性约束？**

线性约束是指可以用以下等式或不等式来表示的约束条件：

*   `线性表达式 = 线性表达式`  可以表示为  `ax₁ + bx₂ + ... + c = dx₁ + ex₂ + ... + f`
*   `线性表达式 ≤ 线性表达式`  可以表示为  `ax₁ + bx₂ + ... + c ≤ dx₁ + ex₂ + ... + f`
*   `线性表达式 ≥ 线性表达式`  可以表示为  `ax₁ + bx₂ + ... + c ≥ dx₁ + ex₂ + ... + f`

其中，$x_1, x_2, ...$ 是变量,  $a, b, c, d, e, f ...$ 是常数。

任何线性约束都可以被改写成一个或两个“线性表达式小于等于零”的形式。 例如，  `ax₁ + bx₂ + c ≥ dx₁ + ex₂ + f`  可以改写为  `(d-a)x₁ + (e-b)x₂ + (f-c) ≤ 0`。

**注意：** 严格不等号（即 `>` 和 `<`）不允许出现在线性约束中。 线性约束只能使用 `<=`、`>=` 或 `=` 。

## EXAMPLE1
**电话的建模案例:普通建模,不可行情况与松弛设置**

$
maximize:\\
\ \ 12\ desk\_production + 20\ cell\_production\\
subject\ to: \\
\ \   desk\_production >= 100 \\
\ \   cell\_production >= 100 \\
\ \   0.2\ desk\_production + 0.4\ cell\_production <= 400 \\
\ \   0.5\ desk\_production + 0.4\ cell\_production <= 490 \\
$
* assembly time,paiting time is for 0.2,0.5 of desk_prod
* desk_product 数量应该大于等于100
* cell_product 数量大于等于100
* desk_product  DeskProduction 的组装时间加上 CellProduction 的组装时间不应超过 400 小时。
* desk_product DeskProduction 的喷涂时间加上 CellProduction 的喷涂时间不应超过 490 小时。


In [4]:
from docplex.mp.model import Model 
m = Model(name="demo") # 

In [5]:
desk = m.continuous_var(name="desk",lb=0)  # 定义连续变量desk，表示桌子的生产量
cell = m.continuous_var(name="cell",lb=0)  # 定义连续变量cell，表示手机的生产量
m.add_constraint(desk >= 100)  
m.add_constraint(cell >= 100)  
m.add_constraint(0.2 * desk + 0.4 * cell <= 400)   # assembly time
m.add_constraint(0.5 * desk + 0.4 * cell <= 490)   # paiting time

# 目标表达式
m.maximize(12*desk+20*cell)
m.print_information() # 打印建模信息
# 求解
solution = m.solve()
# 打印求解
print(solution)



Model: demo
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 3
   - linear=3
 - parameters: defaults
 - problem type is: LP
solution for: demo
objective: 20600
desk=300.000
cell=850.000



### Infeasible model 
* Calling `solve()` on an infeasible model returns `None`. 
* `get_var_by_name()`

In [6]:
# make a new model,copy of m
im = m.copy() # 复制模型
# get the "desk" var of the new model from its name
idesk = im.get_var_by_name("desk") # 获取新的引用
# add a new infeasible cons
im.add_constraint(idesk>=1100);
# solve the new proble, we expect a result of None 
im.print_information(); # 打印建模信息
ims = im.solve()
if ims is None:
    print("model is infeasible")
else:
    print(ims)

# help(m.get_var_by_name)

Model: Copy of demo
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - problem type is: LP
model is infeasible


### 松弛设置relaxation
在 LP 模型中，"松弛 "一词指的是改变约束条件的右边，以允许违反原来的约束条件。<br>
**约束**分为**hard constraint**和**soft constraint**:
* 硬约束在任何情况下都不能违反。到目前为止，我们遇到的所有约束都是硬约束。
* 软约束是指在某些情况下可以违反的约束。

#### Converting hard constraints to soft 硬软转换<br>
原始: 
$$0.2*desk+0.4*cell<=400$$
加班40h需要额外费用但是可以承受
$$0.2*desk+0.4*cell<=400+overtime$$
$$overtime<=40$$
在加上增加的惩罚软约束
$$maximize(12*desk+20*cell-2*overtime)$$
2是加班装配的单位时间成本


In [2]:
from docplex.mp.model import Model 
m = Model(name="demo") 

desk = m.continuous_var(name="desk",lb=0)  # 定义连续变量desk，表示桌子的生产量
cell = m.continuous_var(name="cell",lb=0)  # 定义连续变量cell，表示手机的生产量
overtime =m.continuous_var(name="overtime",ub=40)
m.add_constraint(desk >= 100)  
m.add_constraint(cell >= 100)  

m.add_constraint(0.2 * desk + 0.4 * cell <= 400+overtime)   # assembly time
m.add_constraint(0.5 * desk + 0.4 * cell <= 490)   # paiting time

# 目标表达式
m.maximize(12*desk+20*cell-2*overtime) #2是成本惩罚系数
m.print_information(); # 打印建模信息
# 求解
solution = m.solve()
# 打印求解
print(solution)




Model: demo
 - number of variables: 3
   - binary=0, integer=0, continuous=3
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - problem type is: LP
solution for: demo
objective: 22253.3
desk=166.667
cell=1016.667
overtime=40.000

